In [106]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark import SparkConf

In [107]:
import pandas as pd
import numpy as np
from sklearn import grid_search, datasets
from spark_sklearn import GridSearchCV
from sklearn import ensemble
from pyspark.sql import SparkSession
from spark_sklearn.util import createLocalSparkSession
from patsy import dmatrices
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [108]:
df = pd.read_csv("../data/matcheswithfeatures.csv", index_col = 0)
df.tail()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,...,venue,umpire1,umpire2,umpire3,Avg_SR_Difference,Avg_WPR_Difference,Total_MVP_Difference,Prev_Enc_Team1_WinPerc,Total_RF_Difference,team1Winning
510,511,2015,Mumbai,2015-05-16,Rajasthan Royals,Kolkata Knight Riders,Rajasthan Royals,bat,normal,0,...,Brabourne Stadium,RM Deshpande,RK Illingworth,NaN,-3.303823,-0.271935,-16,50.000000,0.000000,1
513,514,2015,Mumbai,2015-05-19,Mumbai Indians,Chennai Super Kings,Mumbai Indians,bat,normal,0,...,Wankhede Stadium,HDPK Dharmasena,RK Illingworth,NaN,6.315981,-0.617777,-24,50.000000,0.000000,1
514,515,2015,Pune,2015-05-20,Royal Challengers Bangalore,Rajasthan Royals,Royal Challengers Bangalore,bat,normal,0,...,Maharashtra Cricket Association Stadium,AK Chaudhary,C Shamshuddin,NaN,-2.200375,0.969143,5,50.000000,0.000000,1
515,516,2015,Ranchi,2015-05-22,Royal Challengers Bangalore,Chennai Super Kings,Chennai Super Kings,field,normal,0,...,JSCA International Stadium Complex,AK Chaudhary,CB Gaffaney,NaN,-0.521025,1.039181,-23,38.888889,33.333333,0
516,517,2015,Kolkata,2015-05-24,Mumbai Indians,Chennai Super Kings,Chennai Super Kings,field,normal,0,...,Eden Gardens,HDPK Dharmasena,RK Illingworth,NaN,-1.575550,-1.707931,-24,52.380952,0.000000,1


In [109]:
spark = createLocalSparkSession()

In [110]:
y, X = dmatrices('team1Winning ~ 0 + Avg_SR_Difference + Avg_WPR_Difference + Total_MVP_Difference + Prev_Enc_Team1_WinPerc + \
                  Total_RF_Difference', df, return_type="dataframe")
y_arr = np.ravel(y)

In [111]:
X.tail()

,Avg_SR_Difference,Avg_WPR_Difference,Total_MVP_Difference,Prev_Enc_Team1_WinPerc,Total_RF_Difference
510,-3.303823,-0.271935,-16.0,50.000000,0.000000
513,6.315981,-0.617777,-24.0,50.000000,0.000000
514,-2.200375,0.969143,5.0,50.000000,0.000000
515,-0.521025,1.039181,-23.0,38.888889,33.333333
516,-1.575550,-1.707931,-24.0,52.380952,0.000000


In [112]:
X_timetrain = X.loc[X.index < 398]
Y_timetrain = y.loc[y.index < 398]
Y_timetrain_arr = np.ravel(Y_timetrain)
X_timetest = X.loc[X.index >= 398]
Y_timetest = y.loc[y.index >= 398]
Y_timetest_arr = np.ravel(Y_timetest)
X_timetest

,Avg_SR_Difference,Avg_WPR_Difference,Total_MVP_Difference,Prev_Enc_Team1_WinPerc,Total_RF_Difference
398,-9.646646,0.466526,6.0,16.666667,0.000000
399,4.963605,0.097800,12.0,50.000000,0.000000
400,7.079810,0.432566,11.0,70.000000,0.000000
402,21.485599,1.176414,17.0,53.846154,-100.000000
403,-4.503334,1.663169,15.0,54.545455,100.000000
404,-7.297630,-0.332117,-1.0,72.727273,-100.000000
405,12.183316,2.316918,5.0,66.666667,-50.000000
407,-5.341707,2.620287,12.0,61.538462,50.000000
408,5.093091,0.588349,20.0,54.545455,-50.000000
410,-13.668459,-2.328697,0.0,60.000000,-66.666667


In [113]:
tuned_parameters = {
    "n_estimators": [ 100 ],
    "max_depth" : [ 3 ],
    "learning_rate": [ 0.1 ],
}
gbc = ensemble.GradientBoostingClassifier()
clf = GridSearchCV(spark.sparkContext, gbc, tuned_parameters)
clf

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100], 'learning_rate': [0.1], 'max_depth': [3]},
       pre_dispatch='2*n_jobs', refit=True,
       sc=<pyspark.context.SparkContext object at 0x7fc96c0d3dd0>,
       scoring=None, verbose=0)

In [114]:
clf.fit(X_timetrain, Y_timetrain_arr)
clftest_pred = clf.predict(X_timetest)
print "Accuracy is ", metrics.accuracy_score(Y_timetest_arr, clftest_pred) *100, "%"

Accuracy is  50.5747126437 %


In [115]:
knn1 = KNeighborsClassifier()
knn_params = {
    "n_neighbors": [31]
}
clf2 = GridSearchCV(spark.sparkContext, knn1, knn_params, n_jobs = 2)
clf2

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=2, param_grid={'n_neighbors': [31]},
       pre_dispatch='2*n_jobs', refit=True,
       sc=<pyspark.context.SparkContext object at 0x7fc96c0d3dd0>,
       scoring=None, verbose=0)

In [116]:
clf2.fit(X_timetrain, Y_timetrain_arr)
clf2test_pred = clf2.predict(X_timetest)
print "Accuracy is ", metrics.accuracy_score(Y_timetest_arr, clf2test_pred) *100, "%"

Accuracy is  64.367816092 %
